# | default_exp 
Convert Epicurious data to Postgresql

In [ ]:
# | hide
# from datetime import datetime
from hashlib import sha256
# from enum import Enum
import json
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.types import ARRAY, String
# from pydantic import BaseModel
# import vespa
# import requests
from typing import List
from urllib.parse import urlparse

pd.options.display.max_colwidth = None

## Notes
For current MySQL database on NAS
- Column translation to Postgres structure
    - 'id' -> 'source_id'
    - 'recipe_title' -> 'title'
    - 'recipe_url' -> 'url'
    - 'recipe_photo' -> 'photo_url'
    - 'description' -> 'description'
    - 'ingredients' -> 'ingredients'
    - 'steps' -> 'steps'
    - 'cuisine' -> 'cuisines'
- Need to add the following columns:
    - 'origin' = 'AllRecipes'
    - 'language' = 'English'
    - 'mealeon_id' = sha256(unique part of the url)
- Need to convert the following columns:
    - 'cuisine': string -> array
    - 'ingredients': string -> array
    - 'steps': string -> array

In [ ]:
# load MySQL credentials from secrets file
mariadb_key_path = '../secrets/pw.json'

with open(mariadb_key_path, 'r') as fo:
    mariadb_key =  json.loads(fo.read())
user = mariadb_key['user']
password = mariadb_key['password']
host = "10.0.0.218"


In [ ]:
# connect to MySQL

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/scraped_recipes?charset=utf8")

with engine.begin() as connection:
    df = pd.read_sql_table(
        table_name='all_recipes_world_cuisines', 
        con=connection, 
        columns=['id', 'recipe_title', 'recipe_url', 'photo', 'description', 'ingredients', 'steps', 'cuisine'],
    )

df.head(2)

,id,recipe_title,recipe_url,photo,description,ingredients,steps,cuisine
0,255989,Spicy Chicken and Hominy Mexican Soup,https://www.allrecipes.com/recipe/255989/spicy-chicken-and-hominy-mexican-soup/,https://images.media-allrecipes.com/userphotos/4418848.jpg,"Colorful, warm and spicy, this soup is destined to get your taste buds going. A wonderful comfort dish for any chilly day. Garnish with sour cream.","1 tablespoon olive oil||2 chicken breasts, cut into 1-inch pieces||1 small onion, chopped||2 chipotle peppers in adobo sauce, seeded and diced||2 cloves garlic, minced||1 pinch garlic salt, or to taste||1 (32 ounce) can enchilada sauce||2 (16 ounce) cans hominy||1 (15 ounce) can diced tomatoes||1 (15 ounce) can black beans, rinsed and drained||1 ½ cups water||2 tablespoons chili powder||1 tablespoon ground cumin||1 teaspoon dried oregano||1 pinch cayenne pepper||salt and ground black pepper to taste||¼ cup chopped cilantro","Heat oil in a large pot over medium-high heat. Add chicken, onion, chipotle peppers, garlic, and garlic salt; cook and stir until lightly browned, 5 to 8 minutes.||Stir enchilada sauce, hominy, tomatoes, black beans, and water into the pot. Season with chili powder, cumin, oregano, cayenne pepper, salt, and pepper. Bring to a gentle boil. Cover and simmer until flavors combine, about 40 minutes. Garnish with cilantro.",Mexican
1,246332,Carnitas - Pressure Cooker,https://www.allrecipes.com/recipe/246332/carnitas-pressure-cooker/,https://images.media-allrecipes.com/userphotos/3367894.jpg,Savory and tender pork carnitas.,"1 (6 pound) pork butt roast||1 ½ tablespoons salt||1 tablespoon dried oregano||2 teaspoons ground cumin||1 teaspoon ground black pepper||½ teaspoon chile powder||½ teaspoon paprika||2 tablespoons olive oil, or more to taste||1 cup orange juice||1 onion, coarsely chopped||4 cloves garlic, diced, or more to taste","Trim excess fat from pork butt; cut pork into 2-inch cubes and transfer to a bowl.||Combine salt, oregano, cumin, black pepper, chile powder, and paprika together in a bowl. Rub pork cubes with spice mixture. Coat seasoned pork cubes lightly in olive oil; place in pressure cooker. Cover pork cubes with orange juice, onion, and garlic.||Place lid on pressure cooker and lock; bring to full pressure over medium heat until pork is no longer pink in the center, about 60 minutes. Let pressure come down naturally, about 15 minutes.||Remove pork from pressure cooker and shred meat.",Mexican


In [ ]:
df.rename(columns={
            'id': 'source_id', 
            'recipe_title':'title', 
            'recipe_url':'url', 
            'photo': 'photo_url', 
            'description': 'description', 
            'ingredients': 'ingredients', 
            'steps': 'steps', 
            'cuisine': 'cuisines'
            }
        , inplace=True
)

df.head(2)

,source_id,title,url,photo_url,description,ingredients,steps,cuisines
0,255989,Spicy Chicken and Hominy Mexican Soup,https://www.allrecipes.com/recipe/255989/spicy-chicken-and-hominy-mexican-soup/,https://images.media-allrecipes.com/userphotos/4418848.jpg,"Colorful, warm and spicy, this soup is destined to get your taste buds going. A wonderful comfort dish for any chilly day. Garnish with sour cream.","1 tablespoon olive oil||2 chicken breasts, cut into 1-inch pieces||1 small onion, chopped||2 chipotle peppers in adobo sauce, seeded and diced||2 cloves garlic, minced||1 pinch garlic salt, or to taste||1 (32 ounce) can enchilada sauce||2 (16 ounce) cans hominy||1 (15 ounce) can diced tomatoes||1 (15 ounce) can black beans, rinsed and drained||1 ½ cups water||2 tablespoons chili powder||1 tablespoon ground cumin||1 teaspoon dried oregano||1 pinch cayenne pepper||salt and ground black pepper to taste||¼ cup chopped cilantro","Heat oil in a large pot over medium-high heat. Add chicken, onion, chipotle peppers, garlic, and garlic salt; cook and stir until lightly browned, 5 to 8 minutes.||Stir enchilada sauce, hominy, tomatoes, black beans, and water into the pot. Season with chili powder, cumin, oregano, cayenne pepper, salt, and pepper. Bring to a gentle boil. Cover and simmer until flavors combine, about 40 minutes. Garnish with cilantro.",Mexican
1,246332,Carnitas - Pressure Cooker,https://www.allrecipes.com/recipe/246332/carnitas-pressure-cooker/,https://images.media-allrecipes.com/userphotos/3367894.jpg,Savory and tender pork carnitas.,"1 (6 pound) pork butt roast||1 ½ tablespoons salt||1 tablespoon dried oregano||2 teaspoons ground cumin||1 teaspoon ground black pepper||½ teaspoon chile powder||½ teaspoon paprika||2 tablespoons olive oil, or more to taste||1 cup orange juice||1 onion, coarsely chopped||4 cloves garlic, diced, or more to taste","Trim excess fat from pork butt; cut pork into 2-inch cubes and transfer to a bowl.||Combine salt, oregano, cumin, black pepper, chile powder, and paprika together in a bowl. Rub pork cubes with spice mixture. Coat seasoned pork cubes lightly in olive oil; place in pressure cooker. Cover pork cubes with orange juice, onion, and garlic.||Place lid on pressure cooker and lock; bring to full pressure over medium heat until pork is no longer pink in the center, about 60 minutes. Let pressure come down naturally, about 15 minutes.||Remove pork from pressure cooker and shred meat.",Mexican


In [ ]:
df['url'][0]

'https://www.allrecipes.com/recipe/255989/spicy-chicken-and-hominy-mexican-soup/'

In [ ]:
df['url'][0][34:]

for url in df['url'][0:20]:
    print(url[34:])

255989/spicy-chicken-and-hominy-mexican-soup/
246332/carnitas-pressure-cooker/
246118/agua-fresca-de-pepino-cucumber-limeade/
244940/drowned-beef-sandwich-with-chipotle-sauce-torta-ahogada/
242252/secret-salsa/
239051/easy-chicken-flautas/
235756/easy-mexican-sopes/
234881/jalapeno-and-cucumber-margarita/
46653/taco-seasoning-i/
14231/guacamole/
232967/fajita-seasoning/
13351/chicken-tortilla-soup-i/
7399/tres-leches-milk-cake/
281802/creamy-white-chicken-chili-with-salsa-verde/
14064/easy-guacamole/
157642/homemade-flour-tortillas/
20680/easy-mexican-casserole/
27072/mexican-rice-ii/
16678/slow-cooker-taco-soup/
7224/pan-de-muertos-mexican-bread-of-the-dead/


In [ ]:
origin = "AllRecipes"
df['origin'] = origin
df['language'] = "English"

In [ ]:
def unique_name_from_str(string: str) -> str:
    """
    Generates a unique id name. Preferably, the input string is the netloc and path of the URL
    """
    return sha256(string.encode("utf8")).hexdigest()

In [ ]:
def mealeon_id_assigner(string:str) -> str:
    """
    Generates a unique id name from original URL pushed through the sha256 hash function
    """
    parsed_url = urlparse(string)
    sha_loc_path = unique_name_from_str(f"{parsed_url.netloc}{parsed_url.path}")

    return f"{origin}-{sha_loc_path}"


In [ ]:
df['mealeon_id'] = df['url'].apply(mealeon_id_assigner)

df.head(2)

,source_id,title,url,photo_url,description,ingredients,steps,cuisines,origin,language,mealeon_id
0,255989,Spicy Chicken and Hominy Mexican Soup,https://www.allrecipes.com/recipe/255989/spicy-chicken-and-hominy-mexican-soup/,https://images.media-allrecipes.com/userphotos/4418848.jpg,"Colorful, warm and spicy, this soup is destined to get your taste buds going. A wonderful comfort dish for any chilly day. Garnish with sour cream.","1 tablespoon olive oil||2 chicken breasts, cut into 1-inch pieces||1 small onion, chopped||2 chipotle peppers in adobo sauce, seeded and diced||2 cloves garlic, minced||1 pinch garlic salt, or to taste||1 (32 ounce) can enchilada sauce||2 (16 ounce) cans hominy||1 (15 ounce) can diced tomatoes||1 (15 ounce) can black beans, rinsed and drained||1 ½ cups water||2 tablespoons chili powder||1 tablespoon ground cumin||1 teaspoon dried oregano||1 pinch cayenne pepper||salt and ground black pepper to taste||¼ cup chopped cilantro","Heat oil in a large pot over medium-high heat. Add chicken, onion, chipotle peppers, garlic, and garlic salt; cook and stir until lightly browned, 5 to 8 minutes.||Stir enchilada sauce, hominy, tomatoes, black beans, and water into the pot. Season with chili powder, cumin, oregano, cayenne pepper, salt, and pepper. Bring to a gentle boil. Cover and simmer until flavors combine, about 40 minutes. Garnish with cilantro.",Mexican,AllRecipes,English,AllRecipes-81d8f52f33f0b0d12de0921a11835fceba3929d2e17ed596b73dff7222c20133
1,246332,Carnitas - Pressure Cooker,https://www.allrecipes.com/recipe/246332/carnitas-pressure-cooker/,https://images.media-allrecipes.com/userphotos/3367894.jpg,Savory and tender pork carnitas.,"1 (6 pound) pork butt roast||1 ½ tablespoons salt||1 tablespoon dried oregano||2 teaspoons ground cumin||1 teaspoon ground black pepper||½ teaspoon chile powder||½ teaspoon paprika||2 tablespoons olive oil, or more to taste||1 cup orange juice||1 onion, coarsely chopped||4 cloves garlic, diced, or more to taste","Trim excess fat from pork butt; cut pork into 2-inch cubes and transfer to a bowl.||Combine salt, oregano, cumin, black pepper, chile powder, and paprika together in a bowl. Rub pork cubes with spice mixture. Coat seasoned pork cubes lightly in olive oil; place in pressure cooker. Cover pork cubes with orange juice, onion, and garlic.||Place lid on pressure cooker and lock; bring to full pressure over medium heat until pork is no longer pink in the center, about 60 minutes. Let pressure come down naturally, about 15 minutes.||Remove pork from pressure cooker and shred meat.",Mexican,AllRecipes,English,AllRecipes-87afe1a9084e67ae620da79673d582c42f5c0865aecab384644c09b0f4231270


In [ ]:
df['cuisines'] = df['cuisines'].str.split()
df['ingredients'] = df['ingredients'].str.split(r'\|\|')
df['steps'] = df['steps'].str.split(r'\|\|')
df.head(2)

,source_id,title,url,photo_url,description,ingredients,steps,cuisines,origin,language,mealeon_id
0,255989,Spicy Chicken and Hominy Mexican Soup,https://www.allrecipes.com/recipe/255989/spicy-chicken-and-hominy-mexican-soup/,https://images.media-allrecipes.com/userphotos/4418848.jpg,"Colorful, warm and spicy, this soup is destined to get your taste buds going. A wonderful comfort dish for any chilly day. Garnish with sour cream.","[1 tablespoon olive oil, 2 chicken breasts, cut into 1-inch pieces, 1 small onion, chopped, 2 chipotle peppers in adobo sauce, seeded and diced, 2 cloves garlic, minced, 1 pinch garlic salt, or to taste, 1 (32 ounce) can enchilada sauce, 2 (16 ounce) cans hominy, 1 (15 ounce) can diced tomatoes, 1 (15 ounce) can black beans, rinsed and drained, 1 ½ cups water, 2 tablespoons chili powder, 1 tablespoon ground cumin, 1 teaspoon dried oregano, 1 pinch cayenne pepper, salt and ground black pepper to taste, ¼ cup chopped cilantro]","[Heat oil in a large pot over medium-high heat. Add chicken, onion, chipotle peppers, garlic, and garlic salt; cook and stir until lightly browned, 5 to 8 minutes., Stir enchilada sauce, hominy, tomatoes, black beans, and water into the pot. Season with chili powder, cumin, oregano, cayenne pepper, salt, and pepper. Bring to a gentle boil. Cover and simmer until flavors combine, about 40 minutes. Garnish with cilantro.]",[Mexican],AllRecipes,English,AllRecipes-81d8f52f33f0b0d12de0921a11835fceba3929d2e17ed596b73dff7222c20133
1,246332,Carnitas - Pressure Cooker,https://www.allrecipes.com/recipe/246332/carnitas-pressure-cooker/,https://images.media-allrecipes.com/userphotos/3367894.jpg,Savory and tender pork carnitas.,"[1 (6 pound) pork butt roast, 1 ½ tablespoons salt, 1 tablespoon dried oregano, 2 teaspoons ground cumin, 1 teaspoon ground black pepper, ½ teaspoon chile powder, ½ teaspoon paprika, 2 tablespoons olive oil, or more to taste, 1 cup orange juice, 1 onion, coarsely chopped, 4 cloves garlic, diced, or more to taste]","[Trim excess fat from pork butt; cut pork into 2-inch cubes and transfer to a bowl., Combine salt, oregano, cumin, black pepper, chile powder, and paprika together in a bowl. Rub pork cubes with spice mixture. Coat seasoned pork cubes lightly in olive oil; place in pressure cooker. Cover pork cubes with orange juice, onion, and garlic., Place lid on pressure cooker and lock; bring to full pressure over medium heat until pork is no longer pink in the center, about 60 minutes. Let pressure come down naturally, about 15 minutes., Remove pork from pressure cooker and shred meat.]",[Mexican],AllRecipes,English,AllRecipes-87afe1a9084e67ae620da79673d582c42f5c0865aecab384644c09b0f4231270


In [ ]:
df['url']

0                         https://www.allrecipes.com/recipe/255989/spicy-chicken-and-hominy-mexican-soup/
1                                      https://www.allrecipes.com/recipe/246332/carnitas-pressure-cooker/
2                        https://www.allrecipes.com/recipe/246118/agua-fresca-de-pepino-cucumber-limeade/
3       https://www.allrecipes.com/recipe/244940/drowned-beef-sandwich-with-chipotle-sauce-torta-ahogada/
4                                                  https://www.allrecipes.com/recipe/242252/secret-salsa/
                                                      ...                                                
4776                                          https://www.allrecipes.com/recipe/272454/curry-orange-swai/
4777                                    https://www.allrecipes.com/recipe/75514/fruited-tofu-curry-salad/
4778                     https://www.allrecipes.com/recipe/281517/taste-of-india-roasted-root-vegetables/
4779                                         h

In [ ]:
df['ingredients'][0:2]

0    [1 tablespoon olive oil, 2  chicken breasts, cut into 1-inch pieces, 1 small onion, chopped, 2  chipotle peppers in adobo sauce, seeded and diced, 2 cloves garlic, minced, 1 pinch garlic salt, or to taste, 1 (32 ounce) can enchilada sauce, 2 (16 ounce) cans hominy, 1 (15 ounce) can diced tomatoes, 1 (15 ounce) can black beans, rinsed and drained, 1 ½ cups water, 2 tablespoons chili powder, 1 tablespoon ground cumin, 1 teaspoon dried oregano, 1 pinch cayenne pepper, salt and ground black pepper to taste, ¼ cup chopped cilantro]
1                                                                                                                                                                                                                             [1 (6 pound) pork butt roast, 1 ½ tablespoons salt, 1 tablespoon dried oregano, 2 teaspoons ground cumin, 1 teaspoon ground black pepper, ½ teaspoon chile powder, ½ teaspoon paprika, 2 tablespoons olive oil, or more to taste, 1 cup orange j

In [ ]:
df['ingredients'][1]

['1 (6 pound) pork butt roast',
 '1 ½ tablespoons salt',
 '1 tablespoon dried oregano',
 '2 teaspoons ground cumin',
 '1 teaspoon ground black pepper',
 '½ teaspoon chile powder',
 '½ teaspoon paprika',
 '2 tablespoons olive oil, or more to taste',
 '1 cup orange juice',
 '1  onion, coarsely chopped',
 '4 cloves garlic, diced, or more to taste']

In [ ]:
df['steps'][0:2]

0                                                                                                                                                                   [Heat oil in a large pot over medium-high heat. Add chicken, onion, chipotle peppers, garlic, and garlic salt; cook and stir until lightly browned, 5 to 8 minutes., Stir enchilada sauce, hominy, tomatoes, black beans, and water into the pot. Season with chili powder, cumin, oregano, cayenne pepper, salt, and pepper. Bring to a gentle boil. Cover and simmer until flavors combine, about 40 minutes. Garnish with cilantro.]
1    [Trim excess fat from pork butt; cut pork into 2-inch cubes and transfer to a bowl., Combine salt, oregano, cumin, black pepper, chile powder, and paprika together in a bowl. Rub pork cubes with spice mixture. Coat seasoned pork cubes lightly in olive oil; place in pressure cooker. Cover pork cubes with orange juice, onion, and garlic., Place lid on pressure cooker and lock; bring to full pressure over m

In [ ]:
df['steps'][1]

['Trim excess fat from pork butt; cut pork into 2-inch cubes and transfer to a bowl.',
 'Combine salt, oregano, cumin, black pepper, chile powder, and paprika together in a bowl. Rub pork cubes with spice mixture. Coat seasoned pork cubes lightly in olive oil; place in pressure cooker. Cover pork cubes with orange juice, onion, and garlic.',
 'Place lid on pressure cooker and lock; bring to full pressure over medium heat until pork is no longer pink in the center, about 60 minutes. Let pressure come down naturally, about 15 minutes.',
 'Remove pork from pressure cooker and shred meat.']

In [ ]:
postgres_key_path = "../secrets/postgres_login.json"
with open(postgres_key_path, "r") as fo:
    postgres_key = json.loads(fo.read())
user = postgres_key["user"]
password = postgres_key["password"]
host = postgres_key["host"]

postgres_engine = create_engine(
    f"postgresql+psycopg://{user}:{password}@{host}/mealeon"
)

with postgres_engine.begin() as connection:
    df.to_sql(name="recipe_scrapes", con=connection, if_exists="fail", index=False, )

ValueError: Table 'recipe_scrapes' already exists.

In [ ]:
postgres_test_query = """WITH staging AS (
        SELECT
            rs.mealeon_id
            , c.cuisine
        FROM recipe_scrapes rs
        CROSS JOIN UNNEST(rs.cuisines) AS c(cuisine)
    )

    SELECT
        cuisine
        , COUNT(DISTINCT mealeon_id) AS num_recipes
    FROM staging
    GROUP BY
        cuisine;"""

from sqlalchemy import text

with postgres_engine.connect() as connection:
    result = connection.execute(text(postgres_test_query))
    for row in result:
        print(f"cuisine: {row.cuisine}, count: {row.num_recipes}",)

cuisine: AFRICAN CARIBBEAN, count: 6
cuisine: African, count: 197
cuisine: African-American, count: 1
cuisine: African/Carribean, count: 3
cuisine: American, count: 638
cuisine: Argentinian, count: 1
cuisine: Asian, count: 29
cuisine: Australian, count: 1
cuisine: Brazilian, count: 4
cuisine: British, count: 18
cuisine: Cajun, count: 11
cuisine: Canadian, count: 2
cuisine: Caribbean, count: 119
cuisine: Carribean, count: 3
cuisine: Chinese, count: 3
cuisine: Creole, count: 6
cuisine: Cuban, count: 3
cuisine: Danish, count: 2
cuisine: English, count: 10
cuisine: Ethiopian, count: 2
cuisine: European, count: 8
cuisine: Filipino, count: 1940
cuisine: French, count: 52
cuisine: Fusion, count: 53
cuisine: German, count: 4
cuisine: Greek, count: 6
cuisine: Hungarian, count: 2
cuisine: Indian, count: 8
cuisine: International, count: 20
cuisine: Irish, count: 7
cuisine: Italian, count: 62
cuisine: Jamaican, count: 20
cuisine: Japanese, count: 3
cuisine: Jewish, count: 2
cuisine: Korean, count:

In [ ]:
with postgres_engine.connect() as connection:
    result = connection.execute(text('SELECT origin, COUNT(DISTINCT mealeon_id) AS num_recipes FROM recipe_scrapes GROUP BY origin ORDER BY num_recipes DESC;'))
    for row in result:
        print(row)

('Panlasang_Pinoy', 1938)
('AfricanBites', 1491)


In [ ]:
dtype_dict = {
    'cuisines': ARRAY(String),
    'ingredients': ARRAY(String), 
    'steps': ARRAY(String),
    'source_id': String,
    'title': String,
    'url': String,
    'photo_url': String,
    'description': String,
    'origin': String,
    'language': String,
    'mealeon_id': String
}

with postgres_engine.begin() as connection:
    df.to_sql(
        name="recipe_scrapes", 
        con=connection, 
        if_exists="append", 
        index=False, 
        dtype=dtype_dict
    )

In [ ]:
postgres_engine = create_engine(
    f"postgresql+psycopg://{user}:{password}@{host}/mealeon"
)

with postgres_engine.connect() as connection:
    result = connection.execute(text('SELECT origin, COUNT(DISTINCT mealeon_id) AS num_recipes FROM recipe_scrapes GROUP BY origin ORDER BY num_recipes DESC;'))
    for row in result:
        print(row)

('AllRecipes', 4781)
('Panlasang_Pinoy', 1938)
('AfricanBites', 1491)


Not sure why results are blank here, but went into postgres and saw that new recipes from previous AllRecipes scrape was added

In [ ]:
# | hide
nbdev.nbdev_export()